In [1]:
import sys
from pathlib import Path

%load_ext autoreload
%autoreload 2

root_path = Path("/home/olivieri/exp").resolve()
src_path = root_path / "src"
sys.path.append(f"{str(src_path)}")

In [2]:
from IPython.display import Markdown
import pandas as pd
import json
from glob import glob
import os
import numpy as np

from data import *
from path import get_eval_prs_path

# Eval Prompt Assessment

In [3]:
# Setting
BY_MODEL = "LRASPP_MobileNet_V3"
IMAGE_RESIZING_MODE = "letterboxed"
OUTPUT_MODE = "points"

In [4]:
root_prs_path = get_selected_annots_path(BY_MODEL, IMAGE_RESIZING_MODE, OUTPUT_MODE) / "eval_prs" / "llm_judge_assessment"

prs_path = glob(f"{root_prs_path}/*.jsonl")
variations = sorted([os.path.splitext(os.path.basename(pr_p))[0] for pr_p in prs_path])

In [5]:
df_pred_gts = pd.DataFrame()
df_score_gts = pd.DataFrame()
df_pred_prs = pd.DataFrame()
df_score_prs = pd.DataFrame()

for var in variations:

    eval_gts = get_many_eval_gt(BY_MODEL, IMAGE_RESIZING_MODE, OUTPUT_MODE, return_state=False)
    eval_prs = get_many_eval_pr(BY_MODEL, IMAGE_RESIZING_MODE, OUTPUT_MODE, "llm_judge_assessment", var, return_state=False)
    # eval_prs = get_many_item(pr_path, return_state=False)

    _df_all_gts = pd.DataFrame.from_dict(eval_gts).transpose()
    _df_pred_gts = _df_all_gts["pred"].map(lambda x: x if x is None else x == "correct")
    _df_score_gts = _df_all_gts["score"]
    
    _df_all_prs = pd.DataFrame.from_dict(eval_prs).transpose()
    _df_pred_prs = _df_all_prs["pred"].map(lambda x: x if x is None else x == "correct")
    _df_score_prs = _df_all_prs["score"]

    df_pred_gts = pd.concat([df_pred_gts, _df_pred_gts], axis=1)
    df_score_gts = pd.concat([df_score_gts, _df_score_gts], axis=1)
    
    df_pred_prs = pd.concat([df_pred_prs, _df_pred_prs], axis=1)
    df_score_prs = pd.concat([df_score_prs, _df_score_prs], axis=1)

df_pred_prs.columns = variations
df_score_prs.columns = variations
df_pred_gts.columns = variations
df_score_gts.columns = variations

## Pred

In [6]:
pred_acc = df_pred_prs.eq(df_pred_gts).where(df_pred_prs.notna() & df_pred_gts.notna(), None).mean()

## Score

In [7]:
score_MAE = np.abs(df_score_gts - df_score_prs).mean(axis=0)
score_ME = (df_score_gts - df_score_prs).mean(axis=0)

## Overall

In [8]:
score_table = pd.concat([pred_acc, score_MAE, score_ME], axis=1)
score_table.columns = ["pred_acc.", "score MAE", "score ME"]
score_table

,pred_acc.,score MAE,score ME
1_original,0.75,1.1,-0.9
2_incomplet,0.90,0.95,-0.75
3_incomplet+spatial_locs,0.85,1.0,-0.8
4_incomplet+lower_scores,0.90,0.85,-0.65
5_incomplet+simplif,0.80,0.85,-0.45
6_incomplet+lower_scores+simplif,0.80,0.8,-0.3
